In [18]:
import torch
import torch.nn.functional as F
import time

# Dummy data (one hot encoded for simplicity)
sequence = torch.tensor([
    [1, 0, 0],  # "a"
    [0, 1, 0],  # "b"
    [0, 0, 1]   # "c"
], dtype=torch.float32)  # Shape: (sequence_length, features)

query_weights = torch.randn(3, 3)
key_weights = torch.randn(3, 3)
value_weights = torch.randn(3, 3)
mask = torch.triu(torch.ones(3, 3), diagonal=0).bool()  # Adding batch dimension to mask

def attention(query, key, value):
    print("query", query)
    print("key", key)
    print("value", value)

    scores = torch.matmul(query, key.transpose(-2, -1)) / (key.size(-1) ** 0.5)
    probs = F.softmax(scores, dim=-1)
    return torch.matmul(probs, value)

keys = torch.matmul(sequence, key_weights)
values = torch.matmul(sequence, value_weights)

def regular_attention_inference():
    output = []
    start_time = time.time()
    for i in range(sequence.shape[0]):
        query = torch.matmul(sequence[i].unsqueeze(0), query_weights)  # Compute query for the current token
        if i == 0:
            prev_keys = keys[i].unsqueeze(0)
            prev_values = values[i].unsqueeze(0)
        else:
            prev_keys = torch.cat([prev_keys, keys[i].unsqueeze(0)], dim=0)
            prev_values = torch.cat([prev_values, values[i].unsqueeze(0)], dim=0)
        out = attention(query, prev_keys, prev_values)
        output.append(out.squeeze(0))  # Squeeze to remove the single-sample dimension
    total_time = time.time() - start_time
    return torch.stack(output), total_time

output_regular, time_regular = regular_attention_inference()
print("Output with Regular Attention:", output_regular)
print("Inference Time with Regular Attention:", time_regular)


query tensor([[ 1.6124, -0.0901, -2.4144]])
key tensor([[ 0.9136, -0.7799, -1.5167]])
value tensor([[-1.2624,  0.1147, -1.2733]])
query tensor([[1.2150, 0.5750, 0.7063]])
key tensor([[ 0.9136, -0.7799, -1.5167],
        [ 0.1580, -0.4411, -0.5268]])
value tensor([[-1.2624,  0.1147, -1.2733],
        [-0.3234, -1.3464, -0.4045]])
query tensor([[-1.3625, -0.6345,  0.5910]])
key tensor([[ 0.9136, -0.7799, -1.5167],
        [ 0.1580, -0.4411, -0.5268],
        [ 1.1179, -1.2967,  0.9533]])
value tensor([[-1.2624,  0.1147, -1.2733],
        [-0.3234, -1.3464, -0.4045],
        [-0.5266,  0.0110, -0.4037]])
Output with Regular Attention: tensor([[-1.2624,  0.1147, -1.2733],
        [-0.7962, -0.6108, -0.8420],
        [-0.5763, -0.5111, -0.5584]])
Inference Time with Regular Attention: 0.013737201690673828


In [7]:
def masked_attention(query, key, value, mask=None):
    print("query", query)
    print("key", key)
    print("value", value)
    print("mask", mask)

    # Ensure key-value pairs and query are correctly shaped
    key = key.transpose(-1, 0)  # Transpose the last two dimensions of key for matmul
    scores = torch.matmul(query, key) / (key.size(-1) ** 0.5)  # Scale dot product
    print("scores", scores)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, float('-inf'))  # Apply the mask
    probs = F.softmax(scores, dim=-1)  # Softmax over the last dimension

    print("probs", probs, probs.shape)
    print("value", value, value.shape)
    return torch.matmul(probs, value)  # Multiply by value

# Prepare keys and values
keys = torch.matmul(sequence, key_weights)
values = torch.matmul(sequence, value_weights)

print(sequence.shape)
print(key_weights.shape)
print(keys.shape)
print(values.shape)

torch.Size([3, 3])
torch.Size([3, 3])
torch.Size([3, 3])
torch.Size([3, 3])


In [11]:
print(mask[:,0])

tensor([ True, False, False])


In [17]:
keys = torch.matmul(sequence, key_weights)
values = torch.matmul(sequence, value_weights)

def masked_attention_inference_no_loop():
    start_time = time.time()

    # Compute query for all elements
    query = torch.matmul(sequence, query_weights)

    # Apply attention with masking over the full sequence
    out = masked_attention(query, keys, values, mask=mask)
    total_time = time.time() - start_time

    return out, total_time

output_masked_no_loop, time_masked_no_loop = masked_attention_inference_no_loop()
print("Output with Masked Attention (no loop):", output_masked_no_loop)
print("Inference Time with Masked Attention (no loop):", time_masked_no_loop)

query tensor([[ 0.9020,  0.4869,  0.9788],
        [ 1.3756,  0.1241,  0.4788],
        [ 0.3452, -0.8425,  0.9042]])
key tensor([[ 1.6274, -0.6209,  2.1127],
        [-0.6393,  0.5502, -0.4787],
        [ 0.2958,  0.5387, -0.1696]])
value tensor([[-2.3917,  0.3886,  1.0850],
        [ 1.1041,  1.3193,  1.6547],
        [ 0.5917, -0.0856,  0.3307]])
mask tensor([[ True,  True,  True],
        [False,  True,  True],
        [False, False,  True]])
scores tensor([[ 1.8668, -0.4487,  0.2097],
        [ 1.8321, -0.6007,  0.2266],
        [ 1.7293, -0.6449, -0.2916]])
probs tensor([[0.7756, 0.0766, 0.1479],
        [0.0000, 0.3042, 0.6958],
        [0.0000, 0.0000, 1.0000]]) torch.Size([3, 3])
value tensor([[-2.3917,  0.3886,  1.0850],
        [ 1.1041,  1.3193,  1.6547],
        [ 0.5917, -0.0856,  0.3307]]) torch.Size([3, 3])
Output with Masked Attention (no loop): tensor([[-1.6829,  0.3898,  1.0171],
        [ 0.7476,  0.3418,  0.7335],
        [ 0.5917, -0.0856,  0.3307]])
Inference Tim